In [7]:
import sklearn.metrics
import networkx as nx
from glob import glob
import igraph as ig
import numpy as np
import re

In [8]:
def read_graph(path):
    edges = []
    labels = []
    with open(path, 'r') as f:
        next(f)
        next_line = next(f)
        if next_line.startswith('*'):
            for line in f:
                i, j, *_ = re.split('\s+', line.strip())
                i, j = int(i), int(j)
                edge = (i, j)
                edges.append(edge)
        else:
            labels.append(re.split('\s+', next_line.strip())[1])
            for line in f:
                if line.startswith('*'):
                    break
                labels.append(re.split('\s+', line.strip())[1])
            for line in f:
                i, j, *_ = re.split('\s+', line.strip())
                i, j = int(i), int(j)
                edge = (i, j)
                edges.append(edge)
    if labels:
        vertex_attrs = {'label': labels}
    else:
        vertex_attrs = {}
    g = ig.Graph(edges=edges, directed=False, vertex_attrs=vertex_attrs)
    g.vs.select(_degree=0).delete()
    g = g.simplify()
    return g

In [9]:
#ig.plot(communities, mark_groups=True, layout=layout)
for path in glob('../real/*.net'):
    print(path)
    g = read_graph(path)
    leiden = g.community_leiden('modularity', n_iterations=-1)
    print(leiden.modularity)
    multilevel = g.community_multilevel()
    print(multilevel.modularity)
    fastgreedy = g.community_fastgreedy().as_clustering()
    print(fastgreedy.modularity)
    print()

../real\airports.net
0.7097560706214349
0.7050089620718927
0.6624902465629291

../real\cat_cortex_sim.net
0.26609734168301397
0.26609734168301397
0.2604355288969367

../real\celegans_metabolic.net
0.45120707209266864
0.43277073616826706
0.4054972107910379

../real\dolphins.net
0.5276096673391084
0.5213994699576757
0.4954906847039279

../real\emailEU.net
0.41661686406143345
0.41435552416516697
0.34137347599765294

../real\emailURV.net
0.5818517543649749
0.5681883908206941
0.5070244399151751

../real\facebook.net
0.8358079734529854
0.834723679025471
0.7773806408102407

../real\football.net
0.6045695626834571
0.6045695626834571
0.5497406651426808

../real\jazz.net
0.44487125467043975
0.44330819023419904
0.4389078153753829

../real\lastfm.net
0.8183056625846973
0.8136363188151954
0.7966054632778499

../real\PGP.net
0.8865230693345868
0.8823473679583896
0.8524877707879339

../real\zachary.net
0.41978961209730437
0.4188034188034188
0.38067061143984227



In [10]:
g = ig.load('../real/zachary.net')
layout = g.layout_auto()